# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-widgets -q
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.11.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
session_id = "268930"
subscription_id= "9a7511b8-150f-4a58-8528-3e7d50216c31"
resource_group= f"aml-quickstarts-{session_id}"
workspace_name= f"quick-starts-ws-{session_id}"

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'hyperdrive-exp'
project_folder = './capstone'
experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
quick-starts-ws-268930
aml-quickstarts-268930
westeurope
9a7511b8-150f-4a58-8528-3e7d50216c31


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CFLSGJAYC to authenticate.


Name,Workspace,Report Page,Docs Page
hyperdrive-exp,quick-starts-ws-268930,Link to Azure Machine Learning studio,Link to Documentation


In [5]:

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
from azureml.core.dataset import Dataset
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction Dataset"

if key in ws.datasets.keys():  
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/hknguyen20/nd00333-capstone/refs/heads/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,False,Normal,172,False,0.0,Up,False
1,49,F,NAP,160,180,False,Normal,156,False,1.0,Flat,True
2,37,M,ATA,130,283,False,ST,98,False,0.0,Up,False
3,48,F,ASY,138,214,False,Normal,108,True,1.5,Flat,True
4,54,M,NAP,150,195,False,Normal,122,False,0.0,Up,False


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_v2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.estimator import Estimator
import os

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(parameter_space={'--C':uniform(0.5,1.5),'--max_iter':choice(16,32,64,128)})

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(
    name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
# Create a ScriptRunConfig Object as similar to estimator but simpler
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    environment=sklearn_env,compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    primary_metric_name='Accuracy',
    policy=early_termination_policy,
    run_config=src,
    max_total_runs=15,
    max_concurrent_runs=5)

In [21]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
from azureml.widgets import RunDetails
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2d877867-a878-457b-b428-e5287e44e286
Web View: https://ml.azure.com/runs/HD_2d877867-a878-457b-b428-e5287e44e286?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-268930/workspaces/quick-starts-ws-268930&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-10-21T09:43:20.2552208Z][GENERATOR][DEBUG]Sampled 5 jobs from search space 
[2024-10-21T09:43:20.5628031Z][SCHEDULER][INFO]Scheduling job, id='HD_2d877867-a878-457b-b428-e5287e44e286_0' 
[2024-10-21T09:43:20.6371316Z][SCHEDULER][INFO]Scheduling job, id='HD_2d877867-a878-457b-b428-e5287e44e286_1' 
[2024-10-21T09:43:20.6869952Z][SCHEDULER][INFO]Scheduling job, id='HD_2d877867-a878-457b-b428-e5287e44e286_2' 
[2024-10-21T09:43:20.7267644Z][SCHEDULER][INFO]Scheduling job, id='HD_2d877867-a878-457b-b428-e5287e44e286_4' 
[2024-10-21T09:43:20.7281576Z][SCHEDULER][INFO]Scheduling job, id='HD_2d877867-a878-457b-b428-e5287e44e286_3' 
[2024-10-21T09:43:21.8249043Z]

{'runId': 'HD_2d877867-a878-457b-b428-e5287e44e286',
 'target': 'compute-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2024-10-21T09:43:19.3823Z',
 'endTimeUtc': '2024-10-21T09:49:23.751292Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1c687f99-d3f2-4cda-b5cf-de60da340ce0',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_2d877867-a878-457b-b428-e5287e44e286_2',
  'score': '0.8586956521739131',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2d877867-a878-457b-b428-e5287e44e286_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-s

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [23]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [24]:
metrics = best_run.get_metrics()
print(metrics)

{'Regularization Strength:': 0.5237035422953957, 'Max iterations:': 128, 'Accuracy': 0.8586956521739131}


In [25]:
print(best_run)

Run(Experiment: hyperdrive-exp,
Id: HD_2d877867-a878-457b-b428-e5287e44e286_2,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-exp,HD_2d877867-a878-457b-b428-e5287e44e286_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [27]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-10-21-09',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
#TODO: Save the best model
best_run.register_model(model_path="outputs/model.pkl", model_name='best_hyperdrive', properties={'accuracy':metrics['Accuracy']})

Model(workspace=Workspace.create(name='quick-starts-ws-268930', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-268930'), name=best_hyperdrive, id=best_hyperdrive:1, version=1, tags={}, properties={'accuracy': '0.8586956521739131'})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

